In [6]:
%%sh
export PYTHONPATH=/home/mash-intern/mash/cue:${PYTHONPATH}

In [7]:
!pwd

/home/mash-intern/mash/cue/test


In [6]:
data_config = config_utils.load_config(data_config, config_type=config_utils.CONFIG_TYPE.DATA)


[INFO] ========== Data config =========
	bam: ../data/demo/inputs/chr21.small.bam
	fai: ../data/demo/inputs/chr21.small.fa.fai
	chr_names: ['chr21']
	log_level: Info
	n_cpus: 1
	logging_level: INFO
	min_refine_buffer: 2000
	refine_buffer_frac_size: 5
	refine_pair_dist_frac_size: 2
	refine_bp_kernels: [0, 50, 500]
	refine_min_support: 2
	refine_disable: False
	min_pair_support: 2
	min_pair_distance: 4000
	max_pair_distance: 1000000
	scan_target_intervals: True
	stream: True
	view_mode: False
	store_img: False
	empty_annotation: False
	bins_per_block: 8000
	min_sv_len: 4000
	min_qual_score: 50
	bam_type: BAM_TYPE.SHORT
	signal_set: SV_SIGNAL_SET.SHORT
	signal_set_origin: SHORT
	bed: None
	blacklist_bed: None
	signal_vmax: {'RD': 600, 'RD_LOW': 800, 'RD_CLIPPED': 600, 'SM': 200, 'SR_RP': 600, 'LR': 600, 'LLRR': 100, 'RL': 100, 'LLRR_VS_LR': 1}
	signal_mapq: {'RD': 20, 'RD_LOW': 0, 'RD_CLIPPED': 20, 'SM': 20, 'SR_RP': 0, 'LR': 0, 'LLRR': 1, 'RL': 1, 'LLRR_VS_LR': 1}
	bin_size: 750
	interva

In [24]:
# model = models.MultiSVHG(config)
# device = torch.device(config.device)
# model.load_state_dict(torch.load(config.model_path, device))
# model.to(device)


In [22]:
# save model pytorch
torch.save(model, 'cue.pth')
print(type(model))

<class 'models.cue_net.MultiSVHG'>


In [23]:
import torch
model2 = torch.load('/home/mash-intern/mash/cue/test/data/demo/models/cue.pth')
model2.eval()
model2.to(device)
print(type(model2))

# model2

<class 'models.cue_net.MultiSVHG'>


In [9]:
# %%sh
# python call.py --data_config data/demo/data_config.yaml --model_config data/demo/model_config.yaml

#### begin


In [12]:
import sys
import os

# Get the directory containing the notebook file
notebook_dir = os.path.dirname(os.path.abspath('__file__'))

# Append the parent directory of the notebook directory to the Python path
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)

import config_utils as config_utils
import test
import core as core
import argparse
from collections import defaultdict
from torch.utils.data import DataLoader
import torch
import models.cue_net as models
import logging
import seq.io as io
import img.datasets as datasets
from img.refinery import SVKeypointRefinery
import seq.refinery
from img.datasets import SVStreamingDataset
import img.utils as utils
import seq.filters as sv_filters
from seq.aln_index import AlnIndex
from joblib import Parallel, delayed
import seq.utils as seq_utils
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")




In [31]:
data_cf_path = './data/demo/data_config.yaml'
model_config = './data/demo/model_config.yaml'
refine_config = None
if refine_config is not None:
    refine_config = config_utils.load_config(args.refine_config)
given_ground_truth = data_config.bed is not None  # (benchmarking mode)
skip_inference = False
# load the configs
config = config_utils.load_config(model_config, config_type=config_utils.CONFIG_TYPE.TEST)
data_config = config_utils.load_config(data_cf_path, config_type=config_utils.CONFIG_TYPE.DATA)

[INFO]  ========== Model config ==========
	model_path: ./data/demo/models/cue.pth
	n_cpus: 1
	gpu_ids: []
	batch_size: 16
	logging_level: INFO
	report_interval: 50
	n_jobs_per_gpu: 1
	signal_set: SV_SIGNAL_SET.SHORT
	class_set: SV_CLASS_SET.BASIC5ZYG
	num_keypoints: 1
	model_architecture: HG
	image_dim: 256
	sigma: 10
	stride: 4
	heatmap_peak_threshold: 0.4
	pretrained_refinenn_path: None
	config_file: ./data/demo/model_config.yaml
	experiment_dir: /home/mash-intern/mash/cue/test/data/demo
	devices: [device(type='cpu')]
	device: cpu
	log_dir: /home/mash-intern/mash/cue/test/data/demo/logs/
	report_dir: /home/mash-intern/mash/cue/test/data/demo/reports/
	log_file: /home/mash-intern/mash/cue/test/data/demo/logs/main.log
	classes: ['NEG', 'DEL-HOM', 'INV-HOM', 'DUP-HOM', 'DEL-HET', 'INV-HET', 'DUP-HET', 'IDUP-HOM', 'IDUP-HET']
	num_classes: 9
	n_signals: 6
[INFO] ========== Data config =========
	bam: ./data/demo/inputs/chr21.small.bam
	fai: /data/demo/inputs/chr21.small.fa.fai
	chr_name

In [32]:
def call(device, chr_names, uid):
    # runs SV calling on the specified device for the specified list of chromosomes
    # load the pre-trained model on the specified device
    # model = models.MultiSVHG(config)
    model =torch.load(config.model_path, device)
    model.to(device)
    logging.root.setLevel(logging.getLevelName(config.logging_level))
    logging.info("Loaded model: %s on %s" % (config.model_path, str(device)))

    # process each chromosome, loaded as a separate dataset
    for chr_name in chr_names:
        predictions_dir = "%s/predictions/%s.%s/" % (config.report_dir, uid, chr_name)
        Path(predictions_dir).mkdir(parents=True, exist_ok=True)
        aln_index = AlnIndex.generate_or_load(chr_name, data_config)
        dataset = SVStreamingDataset(data_config, interval_size=data_config.interval_size, step_size=data_config.step_size, store=False,
                                     include_chrs=[chr_name], allow_empty=True, aln_index=aln_index)
        data_loader = DataLoader(dataset=dataset, batch_size=config.batch_size, shuffle=False,
                                 collate_fn=datasets.collate_fn)
        logging.info("Generating SV predictions for %s" % chr_name)
        predictions = core.evaluate(model, data_loader, config, device, output_dir=predictions_dir,
                                      collect_data_metrics=True, given_ground_truth=given_ground_truth)
        torch.save(predictions, "%s/predictions.pkl" % predictions_dir)
    return True

In [33]:
def main():

    # ------ Image-based discovery ------
    n_procs = len(config.devices)
    chr_name_chunks, chr_names = seq_utils.partition_chrs(data_config.chr_names, data_config.fai, n_procs)
    logging.info("Running on %d CPUs/GPUs" % n_procs)
    logging.info("Chromosome lists processed by each process: " + str(chr_name_chunks))
    outputs_per_scan = []
    for interval_size in [data_config.interval_size]:
        for step_size in [data_config.step_size]:
            scan_id = len(outputs_per_scan)
            if not skip_inference:
                _ = Parallel(n_jobs=n_procs)(
                    delayed(call)(config.devices[i], chr_name_chunks[i], scan_id) for i in range(n_procs))
            outputs = []
            for chr_name in chr_names:
                predictions_dir = "%s/predictions/0.%s/" % (config.report_dir, chr_name)
                logging.debug("Loading: ", predictions_dir)
                predictions_per_chr = torch.load("%s/predictions.pkl" % predictions_dir)
                outputs.extend(predictions_per_chr)
            outputs_per_scan.append(outputs)

    # ------ Genome-based post-processing ------
    chr_index = io.load_faidx(data_config.fai)
    candidates_per_scan = []
    for outputs in outputs_per_scan:
        candidates = []
        filtered_candidates = []
        for output in outputs:
            svs, filtered_svs = utils.img_to_svs(output, data_config, chr_index)
            candidates.extend(svs)
            filtered_candidates.extend(filtered_svs)
        candidates = sv_filters.filter_svs(candidates, data_config.blacklist_bed, filtered_candidates)
        candidates_per_scan.append(candidates)

    sv_calls = candidates_per_scan[0]
    for i in range(1, len(candidates_per_scan)):
        sv_calls = sv_filters.merge_sv_candidates(sv_calls, candidates_per_scan[i])

    # output candidate SVs (pre-refinement)
    candidate_out_bed_file = "%s/candidate_svs.bed" % config.report_dir
    io.write_bed(candidate_out_bed_file, sv_calls)
    chr2calls = defaultdict(list)
    for sv in sv_calls:
        chr2calls[sv.intervalA.chr_name].append(sv)
    for chr_name in chr_names:
        io.write_bed("%s/candidate_svs.%s.bed" % (config.report_dir, chr_name), chr2calls[chr_name])

    # ------ NN-aided breakpoint refinement ------
    post_process_refined = False
    if refine_config is not None and refine_config.pretrained_model is not None:
        def refine(device, chr_names):
            refinet = models.CueModelConfig(refine_config).get_model()
            refinet.load_state_dict(torch.load(refine_config.pretrained_model, refine_config.device))
            refinet.to(device)
            refinet.eval()
            refinery = SVKeypointRefinery(refinet, device, refine_config.padding, refine_config.image_dim)
            for chr_name in chr_names:
                refinery.bam_index = AlnIndex.generate_or_load(chr_name, refine_config)
                refinery.image_generator = SVStreamingDataset(refine_config, interval_size=None, store=False,
                                                            allow_empty=True, aln_index=refinery.bam_index)
                chr_calls = io.bed2sv_calls("%s/candidate_svs.%s.bed" % (config.report_dir, chr_name))
                for sv_call in chr_calls:
                    refinery.refine_call(sv_call)
                chr_out_bed_file = "%s/refined_svs.%s.bed" % (config.report_dir, chr_name)
                io.write_bed(chr_out_bed_file, chr_calls)
        Parallel(n_jobs=n_procs)(delayed(refine)(chr_name_chunks[i]) for i in range(n_procs))
        post_process_refined = True
    elif not data_config.refine_disable:  # ------ Genome-based breakpoint refinement ------
        def refine(chr_names):
            for chr_name in chr_names:
                chr_calls = io.bed2sv_calls("%s/candidate_svs.%s.bed" % (config.report_dir, chr_name)) 
                os.remove("%s/candidate_svs.%s.bed" % (config.report_dir, chr_name))
                chr_calls = seq.refinery.refine_svs(chr_calls, data_config, chr_index)
                io.write_bed("%s/refined_svs.%s.bed" % (config.report_dir, chr_name), chr_calls)
        Parallel(n_jobs=n_procs)(delayed(refine)(chr_name_chunks[i]) for i in range(n_procs))
        post_process_refined = True
        

    # output candidate SVs (post-refinement)
    if post_process_refined:
        sv_calls_refined = []
        for chr_name in chr_names:
            sv_calls_refined.extend(io.bed2sv_calls("%s/refined_svs.%s.bed" % (config.report_dir, chr_name)))
            os.remove("%s/refined_svs.%s.bed" % (config.report_dir, chr_name))
        candidate_out_bed_file = "%s/refined_svs.bed" % config.report_dir
        io.write_bed(candidate_out_bed_file, sv_calls_refined)

    # ------ IO ------
    # write SV calls to file
    io.bed2vcf(candidate_out_bed_file, "%s/svs.vcf" % config.report_dir, data_config.fai,
            min_score=data_config.min_qual_score, min_len=data_config.min_sv_len)

### contig

In [6]:
import pysam
sample_path = "../data/demo/inputs/chr21.small.bam"
aln_file = pysam.AlignmentFile(sample_path, check_sq=False)
ref_info = aln_file.get_index_statistics()
print(len(ref_info))
ref_info

196


[W::hts_idx_load3] The index file is older than the data file: ../data/demo/inputs/chr21.small.bam.bai


[IndexStats(contig='chr1', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr2', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr3', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr4', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr5', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr6', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr7', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr8', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr9', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr10', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr11', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr12', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr13', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr14', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr15', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr16', mapped=0, unmapped=0, total=0),
 IndexStats(contig='chr17', mapped=0, unmapped=0,